In [ ]:
from io import open
import glob
import os
import unicodedata
import string
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker

from IPython.display import HTML
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.regularizers import l2

### Load Data

In [ ]:
# inspect the data directory
def findFiles(path): 
    return glob.glob(path)

print(findFiles('data/names/*.txt'))

In [ ]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines

def load_data(categories=None):
    names_list = []
    for filename in findFiles('data/names/*.txt'):
        category = os.path.splitext(os.path.basename(filename))[0]
        if not categories or category in categories: 
            names = readLines(filename)
            names_list.extend([(name,category) for name in names])
    df = pd.DataFrame(names_list)
    df.columns = ["name","lang"]
    return df
    

languages = ["English","French","Italian","German","Spanish"]
names = load_data(languages)
names.head()

In [ ]:
maxlen = np.max([len(name) for name in names.name])
print("Maximum name length: ", maxlen)
names.groupby('lang')['name'].count()

### Alphabet

Extract from the list of names all the characters occurring. This gives the basis for constructing a vector space representation of the characters by one-hot-vectors.

Foresee a suitable character for the end of the word, e.g. 'END'.

In [ ]:
### START YOUR CODE

alphabet = ...


### END YOUR CODE
alphabet.append('-') # added for later purpose
len_alphabet = len(alphabet)
char_index = dict((c, i) for i, c in enumerate(alphabet))
print("Size of alphabet: ",len_alphabet)
print(alphabet)

### Vector Representations

In [ ]:
language_to_index = {country:index for index,country in enumerate(names.lang.unique())}
index_to_language = {index:country for index,country in enumerate(names.lang.unique())}

def onehot(i, length):
    v = np.zeros(length);
    v[i] = 1
    return v

def name_representation(name, maxlen):
    ### START YOUR CODE
    name_trunc = str(name)[0:maxlen]
    size = len(char_index)
    vector = [onehot(char_index[j], size) for j in str(name)]
    # fill the rest with 
    for k in range(0,maxlen - len(str(name))):
        vector.append(onehot(char_index['END'], size))
    return vector
    ### START YOUR CODE

def lang_representation(language, language_to_index):
    y = np.zeros(len(language_to_index))
    y[language_to_index[language]]=1
    return y

def lang_from_output(score):
    return index_to_language[np.argmax(score)]

def predict(name, model):
    score = model.predict(np.array([name_representation(name, maxlen)]))[0]
    return lang_from_output(score)

In [ ]:
print(lang_representation("French",language_to_index))

### Prepare train/test

In [ ]:
msk = np.random.rand(len(names)) < 0.8
train = names[msk]
test = names[~msk]

In [ ]:
X_train = []
Y_train = []
for name in train.name:
    X_train.append(name_representation(name,maxlen))
for lang in train.lang:
    Y_train.append(lang_representation(lang, language_to_index))

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
print(X_train.shape,Y_train.shape) 

In [ ]:
X_test = []
Y_test = []
for name in test.name:
    X_test.append(name_representation(name,maxlen))
for lang in test.lang:
    Y_test.append(lang_representation(lang, language_to_index))
        
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)
print(X_test.shape,Y_test.shape) 

### Utility

In [ ]:
def performance_plots(model, X_test, Y_test, log, epochs):    
    acc = log.history['accuracy']
    val_acc = log.history['val_accuracy']
    print('Train accuracy (max):', acc[-1])
    print('Train accuracy (end):', np.max(acc))
    print('Test accuracy (end) :', val_acc[-1])
    print('Test accuracy (end) :', np.max(val_acc))

    plt.plot(log.history['accuracy'])
    plt.plot(log.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.axis([0,epochs,0.0,1.0])
    plt.show()

    plt.plot(log.history['loss'])
    plt.plot(log.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc = 'upper left')
    plt.axis([0,epochs,0.0,2.0])
    plt.show()

### Define and Train the Model

Create an RNN consisting of a single layer with a SimpleRNN (keras) and a softmax.

Then train the model. Play with different number of hidden units in the layer to obtain a good accuracy.

In [ ]:
### START YOUR CODE
### START YOUR CODE
n_languages = 
n_hidden = 

model = 
model.add(SimpleRNN(...))
...

### END YOUR CODE
model.summary()
### END YOUR CODE

In [ ]:
### START YOUR CODE
...


### END YOUR CODE

In [ ]:
performance_plots(model, X_test, Y_test, log, nepochs)

### Confusion matrix

Compute the confusion matrix and jduge for which languages the names are hard to classify. 

In [ ]:
### START YOUR CODE
...

cm = 
### END YOUR CODE
print(cm)

In [ ]:
# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + languages, rotation=90)
ax.set_yticklabels([''] + languages)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()